#### Modelado con SVM (Support Vector Machine)

Importacion de librerias y archivo 

In [9]:
import pandas as pd
import numpy as np
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [10]:
#leer csv
df = pd.read_csv('defunciones_clean.csv')


Quitar columnas no relevantes

In [11]:
# drop columns no relevantes
df.drop('Departamento_Residencia_Madre', axis=1, inplace=True)
df.drop('Año_Registro', axis=1, inplace=True)
df.drop('Mes_Registro', axis=1, inplace=True)
df.drop('Día_Ocurrencia', axis=1, inplace=True)        # No se considera el día de ocurrencia

df.drop('Departamento_Registro', axis=1, inplace=True)

df.drop('Municipio_Registro', axis=1, inplace=True)
df.drop('Municipio_Ocurrencia', axis=1, inplace=True)          
df.drop('Municipio_Residencia_Madre', axis=1, inplace=True)

Separar en entrenaminto y prueba

In [12]:

X = df.drop("Causa_Defunción", axis=1)
Y = df["Causa_Defunción"]

#dividir el dataset en conjunto de entrenamiento y conjunto de testing
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, Y, test_size = 0.25, random_state = 100)


Se utiiza GridSearchCV para determinar que hiperparámetros son los más óptimos para el respectivo modelo.

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Split the dataset into training and testing sets
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, Y, test_size=0.25, random_state=100)

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 'scale']
}

# Create an SVM classifier
svm = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5)
grid_search.fit(X_entreno, y_entreno)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the SVM model with the best parameters
svm_best = SVC(**best_params)
svm_best.fit(X_entreno, y_entreno)

# Make predictions on the testing set
y_pred = svm_best.predict(X_prueba)

# Evaluate the model
classification_rep = classification_report(y_prueba, y_pred)
print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Classification Report:\n", classification_rep)
